# Thinking

## 如何使用用户标签来指导业务（如何提升业务）
    可以使用用户标签实现精准营销，分析产品的潜在用户和用户的潜在需求，构建搜索引擎，广告投放系统来提升服务精准度，理解用户使用产品的心理动机和行为习惯，完善产品运营，提升服务质量，达到产品获客、黏客、留客目的

## 如果给你一堆用户数据，没有打标签。你该如何处理

  1. 可以找一些专家对其中部分数据打标签，然后通过机器学习分类方法对剩余数据打标签打标签
  2. 通过系统生成一批问卷调查，让用户对这批数据打标签
  3. 通过聚类方法设定标签数量，来对数据自动打标签


## 准确率和精确率有何不同（评估指标）
   准确率是混合矩阵中对数据预测正确的概率，它是结果为1的正确预测概率和结果为0的正确预测概率之和，用于表示被分对的样本数除以所有的样本数
   精确率是混合矩阵中对正向目标数据中预测正确的概率，它是结果为1的样本数占实际样本数量的概率 用于表示在被所有预测为正的样本中实际为正样本的概率

## 如果你使用大众点评，想要给某个餐厅打标签。这时系统可以自动提示一些标签，你会如何设计（标签推荐）
    当该餐厅是新餐厅的时候，可以给用户u推荐整个系统最热门的标签或者推荐他自己经常使用的标签
    当用户是没用使用过标签的时候给用户u推荐物品i上最热门的标签
    如果餐厅有标签，用户也打过标签，可以使用将餐厅标签与用户自己的标签融合加权方式，对最终权重大的标签推荐


## 我们今天使用了10种方式来解MNIST，这些方法有何不同？你还有其他方法来解决MNIST识别问题么（分类方法）

    Logistic Regression :逻辑回归模型，可解释性非常好
    CART，ID3（决策树）：树模型，同增同减操作，所以不需要归一化和one-hot操作
    朴素贝叶斯：假设各个特征彼此独立的贝叶斯概率
    SVM：基于训练集合D DD在样本空间中找到一个划分超平面，将不同的类别样本分开
    KNN：通过测量不同特征值之间的距离进行分类，如果一个样本在特征空间中的k个最相似(即特征空间中最邻近)的样本中的大多数属于某一个类别，
         则该样本也属于这个类别
    Adaboost: 迭代算法，针对同一个训练集训练不同的分类器(弱分类器)，然后把这些弱分类器集合起来，构成一个更强的最终分类器（强分类器）
    XGBoost: Boost算法一个成员根本思想在于通过多个简单的弱分类器通过不断加入特征新的树最快速度降低残差，构建出准确率很高的强分类器
    TPOT：通过遗传算法进行特征选择和算法模型选择，自动完成机器学习流程设计中特征工程、模型选择、超参优化三个重要环节，
          以此来生成机器学习的主线代码
    keras：是神经网络模型算法，通过深度计算方式来实现

    分类方法主要包括：随机森林 朴素贝叶斯  决策树 LDA等

# Action

## 针对Delicious数据集，对SimpleTagBased算法进行改进（使用NormTagBased、TagBased-TFIDF算法）

In [8]:
# 使用SimpleTagBased算法对Delicious2K数据进行推荐
# 原始数据集：https://grouplens.org/datasets/hetrec-2011/
# 数据格式：userID     bookmarkID     tagID     timestamp
import random
import math
import operator
import pandas as pd

file_path = "./code/delicious-2k/user_taggedbookmarks-timestamps.dat"
# 字典类型，保存了user对item的tag，即{userid: {item1:[tag1, tag2], ...}}
records = {}
# 训练集，测试集
train_data = dict()
test_data = dict()
# 用户标签，商品标签
user_tags = dict()
tag_items = dict()
user_items = dict()

# 使用测试集，计算准确率和召回率
def precisionAndRecall(N,methods="recommend"):
    hit = 0
    h_recall = 0
    h_precision = 0
    for user,items in test_data.items():
        if user not in train_data:
            continue
        # 获取Top-N推荐列表
        # 通过globals()返回的字典查找函数方法，来获取排名
        rank = globals()[methods](user, N)

#         rank = recommend(user, N)
        for item,rui in rank:
            if item in items:
                hit = hit + 1
        h_recall = h_recall + len(items)
        h_precision = h_precision + N
    #print('一共命中 %d 个, 一共推荐 %d 个, 用户设置tag总数 %d 个' %(hit, h_precision, h_recall))
    # 返回准确率 和 召回率
    return (hit/(h_precision*1.0)), (hit/(h_recall*1.0))

# 对用户user推荐Top-N
def recommend(user, N):
    recommend_items=dict()
    # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和
    tagged_items = user_items[user]     
    for tag, wut in user_tags[user].items():
        #print(self.user_tags[user].items())
        for item, wti in tag_items[tag].items():
            if item in tagged_items:
                continue
            #print('wut = %s, wti = %s' %(wut, wti))
            if item not in recommend_items:
                recommend_items[item] = wut * wti
            else:
                recommend_items[item] = recommend_items[item] + wut * wti
    return sorted(recommend_items.items(), key=operator.itemgetter(1), reverse=True)[0:N]

#NormTagBased算法： score(u,i)= user_tags[u,t]/user_tags[u] * tag_items[t,i]/tag_items[t]
# 对score进行归一化
def recommend_normalTagBased(user, N):
    recommend_items=dict()
    # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和
    tagged_items = user_items[user]     
    for tag, wut in user_tags[user].items():
        #print(self.user_tags[user].items())
        for item, wti in tag_items[tag].items():
            if item in tagged_items:
                continue
            norm = len(user_items[user].items())*len(tag_items[tag].items())
            #print('wut = %s, wti = %s' %(wut, wti))
            if item not in recommend_items:
                recommend_items[item] = wut * wti/norm
            else:
                recommend_items[item] = recommend_items[item] + wut * wti
            
    return sorted(recommend_items.items(), key=operator.itemgetter(1), reverse=True)[0:N]

# 使用测试集，对推荐结果进行评估
def testRecommend(methods="recommend"):
    print("{1}推荐结果评估".format(methods))
    print("%3s %10s %10s" % ('N',"精确率",'召回率'))
    for n in [5,10,20,40,60,80,100]:
        precision,recall = precisionAndRecall(n,methods)
        print("%3d %10.3f%% %10.3f%%" % (n, precision * 100, recall * 100))
        
# 数据加载
def load_data():
    print("开始数据加载...")
    df = pd.read_csv(file_path, sep='\t')
    for i in range(len(df)):
        uid = df['userID'][i]
        iid = df['bookmarkID'][i]
        tag = df['tagID'][i]
        # 键不存在时，设置默认值{}
        records.setdefault(uid,{})
        records[uid].setdefault(iid,[])
        records[uid][iid].append(tag)
    print("数据集大小为 %d." % (len(df)))
    print("设置tag的人数 %d." % (len(records)))
    print("数据加载完成\n")

# 将数据集拆分为训练集和测试集
def train_test_split(ratio, seed=100):
    random.seed(seed)
    for u in records.keys():
        for i in records[u].keys():
            # ratio比例设置为测试集
            if random.random()<ratio:
                test_data.setdefault(u,{})
                test_data[u].setdefault(i,[])
                for t in records[u][i]:
                    test_data[u][i].append(t)
            else:
                train_data.setdefault(u,{})
                train_data[u].setdefault(i,[])
                for t in records[u][i]:
                    train_data[u][i].append(t)        
    print("训练集样本数 %d, 测试集样本数 %d" % (len(train_data),len(test_data)))

# 设置矩阵 mat[index, item] = 1
def addValueToMat(mat, index, item, value=1):
    if index not in mat:
        mat.setdefault(index,{})
        mat[index].setdefault(item,value)
    else:
        if item not in mat[index]:
            mat[index][item] = value
        else:
            mat[index][item] += value


# 使用训练集，初始化user_tags, tag_items, user_items
def initStat():
    records=train_data
    for u,items in records.items():
        for i,tags in items.items():
            for tag in tags:
                #print tag
                # 用户和tag的关系
                addValueToMat(user_tags, u, tag, 1)
                # tag和item的关系
                addValueToMat(tag_items, tag, i, 1)
                # 用户和item的关系
                addValueToMat(user_items, u, i, 1)
    print("user_tags, tag_items, user_items初始化完成.")
    print("user_tags大小 %d, tag_items大小 %d, user_items大小 %d" % (len(user_tags), len(tag_items), len(user_items)))

# 数据加载
load_data()
# 训练集，测试集拆分，20%测试集
train_test_split(0.2)
initStat()
testRecommend()
testRecommend('recommend_normalTagBased')


开始数据加载...
数据集大小为 437593.
设置tag的人数 1867.
数据加载完成

训练集样本数 1860, 测试集样本数 1793
user_tags, tag_items, user_items初始化完成.
user_tags大小 1860, tag_items大小 36884, user_items大小 1860


IndexError: Replacement index 1 out of range for positional args tuple

## 对Titanic数据进行清洗，建模并对乘客生存进行预测。使用之前介绍过的10种模型中的至少2种（包括TPOT）